# `Writer`, `Reader`, and `State` Monads

## Outline

* Introduction
* Logs `Writer`
  - Incentive
  - Definining the instance
  - Helper functions
* Context `Reader`
  - Incentive
  - Definining the instance
  - Helper functions
* `State` manipulator
  - Incentive
  - Definining the instance
  - Helper functions
* `Monad` Transformers (megatron emoji)

In this lesson, we will learn about...

# Logs `Writer`

This one is going to be super easy because we did most of ther work in the previous lesson.
The `Writer` monad represents the effect of writting some information alongside your main computation

## Incentive

In the previous lesson we defined the `Log` type like this:

In [ ]:
data Log a = Log [String] a deriving (Show, Eq)

And then, we defined its `Functor`, `Applicative`, and `Monad` instance from this definition plus what we chose the effect to mean:

In [ ]:
instance Functor Log where
  fmap f (Log logs a) = Log logs (f a)

instance Applicative Log where
  pure = Log []
  (Log l1 f) <*> (Log l2 a) = Log (l1 ++ l2) (f a)

instance Monad Log where
    (Log l1 a) >>= k =
      case k a of
        Log l2 b -> Log (l1 ++ l2) b

But, there's one thing that is bothering me about this. We use a list of `Strings` as logs. and we combine them using the `++` function. However, there's no good reason to specialize this type and type class to lists when we already have the `Monoid` type class!

All we need is a value for the concept of empty, which is what `mempty` from `Monoid` represents, and an associative operation to combine them, which is what `<>` from `Semigroup` represents.

So, instead of locking our logs implementation to a specific data type, such as lists, doesn't provide any advantages but it does provide the disadvantages that now I can't use a different structure for the logs. Maybe a tree-like structure would've been better for my usecase, for example.

So, let's generalize our `Log` implementation a bit more.

## Defining the instance

We'll define a new type called `Writer` that will generalize the logs from a list of `Strings` to anything that is an instance of `Monoid`:

In [ ]:
-- data Log a = Log [String] a

-- data Writer a = Writer [String] a

-- data Writer w a = Writer w a

-- newtype Writer w a = Writer (a, w)

newtype Writer w a = Writer { runWriter :: (a, w)} deriving (Show, Eq)

Here I laid out the transformation from our original `Log` type into the final `Writer` type:
1. The first change we're going to do is to rename the type and constructor to `Writer`.
2. Then, because we could use anything that is an instance of `Monoid` as logs, we have to add the type variable `w` for whatever type the log will be, and we replace the list of `String`s with a value `w`. 
3. Then, instead of using two fields, we'll combine the value and logs into a signle tuple, which allows us to use `newtype` instead of `data`. We also swap `a` and `w` at the value level tuple.
4. And, finally, because we need an easy way to easily extract the wrapped value without pattern-matching, we use record-syntax and we name the tuple `runWriter`. The `run` before the typeclass name is a convention that you'll see often. It reads nicely when using the instance.

Now that we have our datatype, let's start creating the instances we need. We'll derive the `Show` and `Eq` instances to esily check the laws and print values. Starting, of course, with the `Functor` instance:

In [ ]:
instance Functor (Writer w) where
  fmap f (Writer (x, w)) = Writer (f x, w)

-- Check Functor law
fmap id (Writer (1, "hello")) == id (Writer (1, "hello"))

The `Functor` instance is pretty straightforward. We patternmatch to access the inner tuple, apply the function to the value while avoiding modifying the structure, and that's it. 

And, as you can see, our instance follows the `Functor` laws.

Now let's define the `Applicative` instance:

In [ ]:
instance Monoid w => Applicative (Writer w) where
    pure x = Writer (x, mempty)
    Writer (f, w1) <*> Writer (x, w2) = Writer (f x, w1 <> w2)

This one is a bit more complex, but I hope still intuitive.

The pure function has to lift the value to but produce no effect. So, we create a new `Writer` value holding `x` but with a `mempty` log. Because we use `mempty`, we have to constraing the type of `w` to be an instance of `Monoid`.

For the `<*>` function, we pattern match both parameters to extract the internal values. Then, we create a new `Writer` value where the value it holds is the result of applying the function of the first parameter to the value of the second and the writer (or logs) is the result of combining both writers in the correct order using the `<>` binary operator. We don't need to add the `Semigroup` constrant because its implicit with the `Monoid` constraint.

Now, let's check if we're following the `Applicative` laws:

In [ ]:
valueW :: Writer [String] Int = Writer (6, ["some value"])
timesTwoW = Writer ((*2), ["times two"])
showW = Writer (show, ["show it"])

(pure id <*> valueW) == valueW -- Identity

(pure show <*> pure 3) == (pure (show 3) :: Writer String String) -- Homomorphism

((pure (.) <*> showW <*> timesTwoW) <*> valueW) == (showW <*> (timesTwoW <*> valueW)) -- Composition

(timesTwoW <*> pure 3) == (pure (\f -> f 3) <*> timesTwoW) -- Interchange

Everything seems to be in order. Finally, let's create the instance for the `Monad` type class:

In [ ]:
instance Monoid w => Monad (Writer w) where
    Writer (x, w1) >>= k = 
        case k x of
            Writer (y, w2) -> Writer (y, w1 <> w2)

I defined the implementation following the same patterns we followed in the previous lesson. And, this would be a good enough implementation. However, we're evaluating a case expression that will always have a single case. We could write this more simply by pattern-matching that unique expression with a let binding like this:

In [ ]:
instance Monoid w => Monad (Writer w) where
    Writer (x, w1) >>= k = let Writer (y, w2) = k x in Writer (y, w1 <> w2)

These two instances are essentially the same, however the second is more succint and explicit about `Writer` being a single-constructor wrapper around tuple. You could also have done the same with a `where` clause.

Now, let's quickly check the monad laws:

In [ ]:
m = Writer (5, ["initial"])
k x = Writer (x*2, [show x ++ " times two"])
h x = Writer (x + 3, [show x ++ " plus three"])

(return 3 >>= k) == k 3  -- left identity
(m >>= return) == m  -- right identity
(m >>= (\x -> k x >>= h)) == ((m >>= k) >>= h) -- associativity

Awesome! It follows the monad laws, so it seems we have a valid `Writer` instance.

## Helper functions

Of course, even thought now we have available all the helper functions relalted to `Functor`, `Applicative`, and `Monad`, nothing stops us from creating our own helper functions specialized to this type.

Let's see an example of how we could potentially use this instance:

In [ ]:
timesTwoW x = Writer (x*2, [show x ++ " times two"])
plusThreeW x = Writer (x + 3, [show x ++ " plus three"])
showW x = Writer (show x, ["show it"])

example :: Int -> Writer [String] String
example x = do
  y <- timesTwoW x
  z <- plusThreeW y
  showW b


runWriter $ example 3

Here we see three functions that manipulate values in various ways. Then, in the `example` funciton, I take an `Int` and apply these monadic functions one at a time to get the final value.

This works well, but it does two things wrong in the first three functions:
- It surfaces the effect at the logic level, which, of course, is what we are trying to avoid by abstracting away the effect.
- And it exposes the internal representation of the `Writer` type that I migh not have available or might be extremely complicated depending on the type.

So, what we can do, is to create the helper function `tell` that will just write new logs in our monad witout modifying the value:

In [ ]:
tell w = Writer ((), w)

And now, we can write the same functions like this:

In [ ]:
timesTwoW x = do
   tell [show x ++ " times two"]
   pure $ x*2

plusThreeW x = do
   tell [show x ++ " plus three"]
   pure $ x + 3

showW x = do
   tell ["show it"]
   pure $ show x

example :: Int -> Writer [String] String
example x = do
  y <- timesTwoW x
  z <- plusThreeW y
  showW b


runWriter $ example 3

And this concludes the generalization of `Log` into the `Writer` type that generalizes into writting any `Monoid` along the result of a computation. It can be used for more things besides logging:
- You can use it to emit intermediate representations in code generating tools
- Output accumulation for domain specific languages, and others.

Also, there are more helper functions besides `tell`. But this is the 80/20. We won't explore more for two reasons:
- One is that I don't want to do a super-long lesson
- The other is that the `Writer` monad has some performance issues, so it's not as widely used as the other two monad we'll learn about today. Speaking of which:

# Context `Reader`

For the `Writer` `Monad`, we could rely on the `Log` `Monad` we developed on the previous lesson, but for the `Reader` `Monad` we have to start from scratch. So, let's do a short setup.

Let's say we're running a website that has a Haskell backend. This website has free and paid users, and we need to define some logic to send emails from our backend. We can start by defining an entry point:

```
main :: IO ()
main = do
  env <- parseEnv <$> readFile ".env"
  sendReminderToPayInvoice env 
```

Becauswe we need to read from our database to get our information about our users, we have to connect to it in some way. However, we don't want to hardcode the address and password of our database in the code, for obvious reasons. So, what do we do? We create an enviromnent file, same as everyone, and read the information from there. That way we don't expose secret information in the source code, and we have have different values for different stages of deployment. So, we create this file:

> Inside `.env` file:
```
DB_HOST=localhost
DB_PORT=5432
DB_PASSWORD=lazyness123
```

Now that we have our values there, we read them using the default `readFile` function, and then we `fmap` the custom `parseEnv` function to trans form plain text into a key-value map:

In [ ]:
import Data.Map (Map, fromList)

parseEnv :: String -> Map String String
parseEnv str = fromList $ map ((\(k, v) -> (k, drop 1 v)) . break (== '=')) (lines str)


parseEnv "KEY_1=value1\nKEY_2=value2"

Once we have those key-value pairs, we can pass them down to other functions. However, the type `Map String String` will always be the same in this case. But it's verbose and lacks the meaning in this particular program, so let's name it `Env` for "environment":

In [ ]:
type Env = Map String String

And since we're defining the domain of our program, and we know we'll deal with users, let's define the user type:

In [ ]:
data User = MkUser
    { userName :: String
    , userEmail :: String
    , isPaid :: Bool
    , hasDebt :: Bool
    }
    deriving (Show)

Now that we have our domain modeled, sort of, we can keep going by defining all the functions that we need. I couldn't run the first code block because the `sendReminderToPayInvoice` function doesn't exist. So, let's define it:

```
sendReminderToPayInvoice :: Env -> IO ()
sendReminderToPayInvoice env = do
    debtors <- getDebtorsInfo env
    print "We sent emails to:"
    print debtors
```

We get all the debtors using the `getDebtorsInfo` action that we'll define after this one, then, instead of actually sending emails, we print a message saying that we did. Because this is obviously a dummy implementation. And after that, we print all the debtors. As you can see, we didn't use the `env` variable yet, because we didn't conect to the database, but we will eventually, so we pass it along to the `getDebtorsInfo` function.

Now, let's go deeper and define that function:

```
getDebtorsInfo :: Env -> IO [User]
getDebtorsInfo env = do
    users <- getUsersInfo env
    pure $ filter (\u -> isPaid u && hasDebt u) users
```

As you can see, the first thing we do is to get a list of all of our users using the `getUsersInfo` action, that takes the `env` variable and returns the users in the database. Once we have the users, we can filter them using the `filter` function and lifting the filtered users again to the `IO` monad before returning because we can't (and shouldn't want to) escape impure actions.

We didn't use `env`, again, but we'll provide it to the next function. We'll need to eventually connect to the database to get the users. So, let's define the `getUsersInfo` action:

```
getUsersInfo :: Env -> IO [User]
getUsersInfo env = do
    let table = "public.users"
    callDatabase table env
```

Okey! Getting warmer! It seems our database is a classic SQL database, and in this function we define the table we want to interact with, the `"public.users"` table, and then we use the more generic `callDatabase` function that take the `table` and the `env` variable and returns a list of `User`s.

Let's define the `callDatabase` function now:

```
callDatabase :: Env -> String -> IO [User]
callDatabase env table = do
    let dbHost = getEnvVar env "DB_HOST"
    let dbPort = getEnvVar env "DB_PORT"
    let dbPass = getEnvVar env "DB_PASSWORD"
    do
        -- Fake call to the database using dbHost, dbPort, and dbPass
        putStrLn $ "Connecting to database at " ++ dbHost ++ ":" ++ dbPort
        putStrLn $ "Using password: " ++ dbPass
        putStrLn $ "Fetching data from table: " ++ table
        pure
            [ MkUser "alice" "alice@example.com" True False
            , MkUser "bob" "bob@example.com" False False
            , MkUser "charlie" "charlie@example.com" True True
            , MkUser "dave" "dave@example.com" False False
            ]
```

There you go!! We do connect to the database in this one! As you can see, we take all the information we need from the environment variable using the `getEnvVar` and the key we want to get the value from. Then, we do a fake call to a database using these values and return a list of four dummy users.

One interesting thing to notice is that we're not really using `env` in this function either. We're just passing it along to the `getEnvVar` function. So, now, let's define our last function:

In [ ]:
import Prelude hiding (lookup)
import Data.Map (lookup)
import Data.Maybe (fromJust)

getEnvVar :: Env -> String -> String
getEnvVar env key = fromJust . lookup key $ env

The `getEnvVar` fuction gets the environment variable as a first argument, and the key of the variable we want to use as the second argument.

Then, we simply `lookup` the `key` within our environment, and extract it from the `Maybe` value, since the lack of this value means that we can't connect to the database, so the application should crash. This is not handle gracefully, but would do for this example.

Now, we have everything we need to run our program:

In [ ]:
{-# OPTIONS_GHC -Wno-unrecognised-pragmas #-}
{-# HLINT ignore "Use second" #-}

module WithReader where

import Data.Map (Map, fromList, lookup)
import Data.Maybe (fromJust)
import Distribution.Compat.Prelude (readMaybe)
import Prelude hiding (lookup)

type Env = Map String String

data User = MkUser
    { userName :: String
    , userEmail :: String
    , isPaid :: Bool
    , hasDebt :: Bool
    }
    deriving (Show)

main :: IO ()
main = do
    env <- parseEnv <$> readFile ".env"
    sendReminderToPayInvoice env

parseEnv :: String -> Env
parseEnv str = fromList $ map ((\(k, v) -> (k, drop 1 v)) . break (== '=')) (lines str)

sendReminderToPayInvoice :: Env -> IO ()
sendReminderToPayInvoice env = do
    debtors <- getDebtorsInfo env
    print "We sent emails to:"
    print debtors

getDebtorsInfo :: Env -> IO [User]
getDebtorsInfo env = do
    users <- getUsersInfo env
    pure $ filter (\u -> isPaid u && hasDebt u) users

getUsersInfo :: Env -> IO [User]
getUsersInfo env = do
    let table = "public.users"
    callDatabase env table

callDatabase :: Env -> String -> IO [User]
callDatabase env table = do
    let dbHost = getEnvVar env "DB_HOST"
    let dbPort = getEnvVar env "DB_PORT"
    let dbPass = getEnvVar env "DB_PASSWORD"
    do
        -- Fake call to the database using dbHost, dbPort, and dbPass
        putStrLn $ "Connecting to database at " ++ dbHost ++ ":" ++ dbPort
        putStrLn $ "Using password: " ++ dbPass
        putStrLn $ "Fetching data from table: " ++ table
        pure
            [ MkUser "alice" "alice@example.com" True False
            , MkUser "bob" "bob@example.com" False False
            , MkUser "charlie" "charlie@example.com" True True
            , MkUser "dave" "dave@example.com" False False
            ]

getEnvVar :: Env -> String -> String
getEnvVar env key = fromJust . lookup key $ env

Let's create the `.env` file first, to have something to work with:

In [ ]:
writeFile ".env" "DB_HOST=localhost\nDB_PORT=5432\nDB_PASSWORD=lazyness123"

Now, we can run "main"

In [ ]:
main

And that's it! We implemented the scafolding to send emails to users that didn't pay on time. This is a dummy implementation, of course, but one notable thing is that, if you look closely, we pass along the `env` variable 5 diferent times before actually using it. This is bad for several reasons:

- A lot of unnecessary noice.
- Creates a rigid hyerarchical structure that is hard to refactor or evolve.
- What if I want to add another value that I only use deep down, like a `debug` flag, or user preferences? Another thing I have to pass through all those functions?

Let's see what we can do about it. If we look at the types of all our functions, we'll see a pattern:

```
sendReminderToPayInvoice :: Env -> IO ()
getDebtorsInfo           :: Env -> IO [User]
getUsersInfo             :: Env -> IO [User]
callDatabase             :: Env -> String -> IO [User]
getEnvVar                :: Env -> String -> String
```

In all cases its a function that takes the `Env` type and returns something else, that could either be a value or a function that takes more arguments. We could represent this concept by defining a generic function that take the environment variable like this:

```
functionWithEnv :: Env -> a
```

Now, how can we capture this in a type? We could have something like this:

```
newtype WithEnv env a = WithEnv (env -> a)
```

In this type, we capture the concept of a function that takes an environment and returns something, and we wrapp it around a type constructor. We also provide both the environment and the return value as type parameters to have full flexibility on to which types we could use.

The names seem a bit constrained, though. Let's go with the default definition everyone uses:

In [ ]:
newtype Reader r a = Reader { runReader :: r -> a}

We just renamed the type variables and changed the constructor to be a record type. That way we can unwrap the function without pattern matching. We call the type `Reader` because we're "reading" the value from somewhere. Following that logic, the first parameter `r` is the value read, and the second parameter `a` is whatever the function returns once we provided `r`.

Now, whenever we see the `Reader r a` type somehwere, we know there's this `r` value available for us. And, of course, this value doesn't appear out of nowhere. What is really happening under the hood is that we're asking for that `r` all the way up to the first function that uses this type. And that's where we have to provide the actual value. If that didn't click, it'll become clear when we start using it.

Now that we have the type, let's implement `Functor`, `Applicative`, and `Monad`. That way we can impicity sequence `Reader` values and extract them whenever we need them. Let's start with the `Functor` instance:

In [ ]:
instance Functor (Reader r) where
--  fmap :: (a -> b) -> Reader r a -> Reader r b
    fmap f (Reader g) = Reader (f . g)

To implement `fmap` we have to understand that the environment variable `r` is part of the structure, so we don't want to modify it. We only modify the actual vale of type `a` that we return at the end. So, we implement the `Functor` instance for the type `Reader r`.

The actual implementation is super simple. We need to return a function of type `r -> b` wrapped in a `Reader` constructor because that's a value of type `Reader r b`. So, we first pattern match to extract the function `g` of type `r -> a`, and, since we have the function `f` of type `a -> b`, we compose them to get a function of type `r -> b` and wrap the result in a `Reader` constructor.

Done! We now have a way to `fmap` `Reader r` values. If you look closely, since the `Reader` constructor just wraps a function, we're just implementing function composition with the extra hassle of unwrapping and wrapping one of the functions in a constructor. Nothing crazy, plus, we just followed the types, once we had the types, almost no brain power is needed to get the implementation.

Now, let's move into implementing the `Applicative` instance:

In [ ]:
instance Applicative (Reader r) where
    -- pure :: a -> Reader r a
    pure x = undefined
    -- Reader r (a -> b) -> Reader r a -> Reader r b
    Reader f <*> Reader g = undefined

We have to implement both `pure` and `<*>`. Let's start with `pure`. The effect of `Reader r` is to read a value and have it available for use within the function. So, how would a `Reader r` value that has no effect look like?

There's just one way to interpret this, we have to ignore the value. So, we can implement `pure` like this:

In [ ]:
instance Applicative (Reader r) where
    -- pure :: a -> Reader r a
    pure x = Reader (\_ -> x) -- Same as: Reader (const x)
    -- Reader r (a -> b) -> Reader r a -> Reader r b
    Reader f <*> Reader g = undefined

If the effect is having the `r` value available, we can lift a pure value by creating a function that ignores the `r` and wrapping it with the `Reader` constructor. That way, the function is independent of the `r` value, hence, no effect is performed.

Moving on to the `<*>` operator, I kid you not, we just need to follow the types again until GHC stops complaining:

In [ ]:
instance Applicative (Reader r) where
    -- pure :: a -> Reader r a
    pure x = Reader (\_ -> x) -- Same as: Reader (const x)
    -- Reader r (a -> b) -> Reader r a -> Reader r b
    Reader f <*> Reader g = undefined

-- Reminder:
-- f :: r -> a -> b
-- g :: r -> a 
-- result needed: Reader wrapping a function of type r -> b

I added the types of the wrapped funcitons and the desired result below as a reminder, but if you try to implement it yourself, GHC will make sure you respect those.

Same as always, we pattern match to extract the value inside the `Reader` constructors. Because we know `f` is a function that is suposed to be applied to `g`, the only possible types for them are `f :: r -> a -> b`, because we need a function `a -> b`, and since it's wrapped inside a `Reader r`, it has to have `r` as the first paramter, and `g :: r -> a`, for the same reasons as before.

Now, we need to return a `Reader` value that takes an `r` and returns a `b`. The first part is easy: We return a `Reader` constructor that wraps a function that takes an `r`:

In [ ]:

instance Applicative (Reader r) where
    -- pure :: a -> Reader r a
    pure x = Reader (\_ -> x) -- Same as: Reader (const x)
    -- Reader r (a -> b) -> Reader r a -> Reader r b
    Reader f <*> Reader g = Reader (\r -> undefined)

-- Reminder:
-- f :: r -> a -> b
-- g :: r -> a 
-- result needed: Reader wrapping a function of type r -> b

Now, all we have to do is figure a way to combine `f`, `g`, and `r` to get a value of type `b`. 

We know that we want to use `f` since it returns exactly what we need: a value of type `b`. We could also apply it to the first parameter of type `r`, since we already have it as well:

In [ ]:

instance Applicative (Reader r) where
    -- pure :: a -> Reader r a
    pure x = Reader (\_ -> x) -- Same as: Reader (const x)
    -- Reader r (a -> b) -> Reader r a -> Reader r b
    Reader f <*> Reader g = Reader (\r -> f r undefined)

-- Reminder:
-- f :: r -> a -> b
-- g :: r -> a 
-- result needed: Reader wrapping a function of type r -> b

Now, to provide as the second argument, we need a value of type `a`; which we can obtain by applying `g` to `r`, so we do that:

In [ ]:
instance Applicative (Reader r) where
    -- pure :: a -> Reader r a
    pure x = Reader (\_ -> x) -- Same as: Reader (const x)
    -- Reader r (a -> b) -> Reader r a -> Reader r b
    Reader f <*> Reader g = Reader (\r -> f r (g r))

-- Reminder:
-- f :: r -> a -> b
-- g :: r -> a 
-- result needed: Reader wrapping a function of type r -> b

And we're done! We now have the effect of reading a value from the environment! To shorten this lesson as much as possible, I won't check the laws or provide usage examples of the `<*>` operator. However, I'll show you the original example turned into `do` notation once we implement the `Monad` instance. Speaking of which:

In [ ]:

instance Monad (Reader r) where
    -- Reader r a -> (a -> Reader r b) -> Reader r b
    Reader f >>= g = undefined

-- Reminder:
-- f :: r -> a
-- result needed: Reader wrapping a function of type r -> b

We need to implement the `>>=` operator. We can pattern match the first parameter to extract the interal function `f`, and from then, we can follow the types same as the previous examples.

Since we need to returns a `Reader r b` type, we can start by wrapping the result of our work in the `Reader` constructor containing a function that takes `r` and somehow produces a `b`:

In [ ]:

instance Monad (Reader r) where
    -- Reader r a -> (a -> Reader r b) -> Reader r b
    Reader f >>= g = Reader (\r -> undefined)

-- Reminder:
-- f :: r -> a
-- result needed: Reader wrapping a function of type r -> b

We know that the only way of getting a value of type `b` is to somehow running the effect `Reader r b` that is returned by `g` when applied to a value of type `a`. So, there's some work to do. We know we want to use the effect returned by `g`, so we'll have to somewhow provide it a value of type `a`. We can obtain a value of that type by applying `f` to `r`, so we'll do just that:

In [ ]:
```
instance Monad (Reader r) where
    -- Reader r a -> (a -> Reader r b) -> Reader r b
    Reader f >>= g = Reader (\r -> g (f r)) -- Won't compile

-- Reminder:
-- f :: r -> a
-- result needed: Reader wrapping a function of type r -> b
```

I have to use a markdown cell because this won't compile. But now we're closer to our objective. Now, we have a value of type `Reader r b` inside anoter `Reader r` value. However, what we need is the actual value `b` returned after the effect, so what we do is running the effect! We have the `runReader` function to extract the inner function from the constructor, so we'll do that:

In [ ]:

instance Monad (Reader r) where
    -- Reader r a -> (a -> Reader r b) -> Reader r b
    Reader f >>= g = Reader (\r -> runReader (g (f r)) undefined) -- Won't compile

-- Reminder:
-- f :: r -> a
-- result needed: Reader wrapping a function of type r -> b

We're almost there. Now we have a function that takes a value of type `r` and returns a value of type `b`, but we just need the `b`, so we apply the resulting funciton of running `runReader` to `r` to get the final expression:

In [ ]:
instance Monad (Reader r) where
    -- Reader r a -> (a -> Reader r b) -> Reader r b
    Reader f >>= g = Reader (\r -> runReader (g (f r)) r)

-- Reminder:
-- f :: r -> a
-- result needed: Reader wrapping a function of type r -> b

And we're done! The expression might look a bit criptic, but if you follow the types, everything clicks like brand new Legos.

You might notice that we haven't checked any of the `Functor`, `Applicative`, and `Monad` laws. I'll trust you'll do it as homework to make sure I'm not deceiving you into thinking these are correct implementations when in reality they are not.

Now that wee have our new `Reader` monad, we can refactor our old code with it! 

Let's start with the entry point. This is our new `main` function:

```
main :: IO ()
main = do
  env <- parseEnv <$> readFile ".env"
  runReader sendReminderToPayInvoice env
```

The only change is that we added the `runReader` function at the beginning of the last line. If you remember correctly, the type of `runReader` is:

```
runReader :: Reader r a -> r -> a
```

So, if we provide a `Reader r a` as first paramater, the `runReader` function will extract the inner function of type `r -> a`, which applied to a value of type `r` will return the final value of type `a`.

We provide the `sendReminderToPayInvoice` function as the value of type `Reader r a` and the `env` value as value of type `r`. From now on, all the functions inside the `sendReminderktoPayInvoice` function will have the `env` value available to them.

Another thing we'll do is to change the `Env` type synonym:

In [ ]:
-- Used to be: type Env = Map String String

type Env = Reader (Map String String)

Since we're inisde the `Reader` monad, and we provided the `env` value of type `Map String String`, we'd have to write that huge type on every type signature. This gets old quickly, so we'll add this type synonym to signal that the `Env` type is a `Reader` monad that provides a value of type `Map String String` as environment variable whenever needed. We don't specify the type of the second parameter because it continuously changes throught each function.

Now that we have that, let's quickly refactor each function following the compiler's messages:

```
sendReminderToPayInvoice :: Env (IO ())
sendReminderToPayInvoice = do
    debtors <- getDebtorsInfo -- debtors :: IO Users
    pure $ do
        print "We sent emails to:"
        debtors >>= print
```

Now the `sendReminderToPayInvoice` function is of type `Env` that returns an `IO ()` as result.

Since we are insind the `Reader` monad, we can run the `getDebtorsInfo` function to get the final `debtors`. We don't have to provide anything to the `getDebtorsInfo` because it already has access to `env` by virtue of being a function inside the `Env` `Reader` monad.

Something that can be tricky to work with are nested monads. In this case, the `debtors` value is pure in regarding the `Reader` monad but inpure regarding the `IO` monad since it's a value of type `IO [Users]`. So, we need another `do` block inside the initial one to work within the `IO` monad. And since we can't return values outside the `Reader` monad, we'll lift that whole `IO` `do` block using the `Reader` monad's `pure` we just defined.

Now that we're inside the `IO` `do` block, we can do `IO` stuff like print, send emails, and lauch misiles.

Moving on to updating the next two functions we have:

```
getDebtorsInfo :: Env (IO [User])
getDebtorsInfo = do
    users <- getUsersInfo
    pure $ filter (\u -> isPaid u && hasDebt u) <$> users

getUsersInfo :: Env (IO [User])
getUsersInfo = do
    let table = "public.users"
    callDatabase table
```

The `getDebtorsInfo` function now doesn't take any parameters and runs the `getUsersInfo` to get the users, which that also doesn't take any parameters.

Since the list of `User`s is inisde the `IO` monad, we have to `fmap` the filter function to get a value of type `IO [User]` back and then use `pure` to lift the value to the `Env` type.

On the `getUsersInfo` side, the only change was removing the now unnecessary parameter and changing the type, everything else remains the same.

Moving into the `callDatabase` function, the change is also quite minimal:

```
callDatabase :: String -> Env (IO [User])
callDatabase table = do
    dbHost <- getEnvVar "DB_HOST"
    dbPort <- getEnvVar "DB_PORT"
    dbPass <- getEnvVar "DB_PASSWORD"
    pure $ do
        -- Fake call to the database using dbHost, dbPort, and dbPass
        putStrLn $ "Connecting to database at " ++ dbHost ++ ":" ++ dbPort
        putStrLn $ "Using password: " ++ dbPass
        putStrLn $ "Fetching data from table: " ++ table
        pure
            [ MkUser "alice" "alice@example.com" True False
            , MkUser "bob" "bob@example.com" False False
            , MkUser "charlie" "charlie@example.com" True True
            , MkUser "dave" "dave@example.com" False False
            ]
```

We removed the unneded parameter and instead of `let` bindings to get the environment values, we have to run them so we can access the environment variable.

In this case we also do some stuff at the `IO` level, so we have to again do the `IO` `do` block within the `Reader` `do` block to be able to run `IO` effects in sequence. This is the second example of sequencing two different monads in the same function body, however they are quite simple and it can get complicated when we start working with many stacked monads. After this course, make sure to explore how to combine effects using the monad transformers and similar libraries.

Finally, the last function we have to refactor is the `getEnvVar` function, which now take a single parameter, the name of the `key` we want to search from:

```
getEnvVar :: String -> Env String
getEnvVar var = undefined
```

However, now we're in a pickle. Even though the `Reader` monad implicitily passes the environment, now we actually need to use the value!

Think about it. How could we do this? 

Well, we know that any value of type `Reader r` receives the `r` value as parameter, so all we have to do is to return it untouched! like this:

```
getEnvVar :: String -> Env String
getEnvVar var = do
    env <- Reader (\r -> r)
    undefined
```

And now that we have the value, we can use it same as always:

```
getEnvVar :: String -> Env String
getEnvVar var = do
    env <- Reader (\r -> r)
    pure . fromJust . lookup var $ env
```

That could be it. However, the behavior of getting the value from the environment is so common that it has a name, and it's called ask:

In [ ]:
ask :: Reader r r
ask = Reader id -- Same as: Reader (\r -> r)

And with that, the final expression for `getEnvVar` is:

```
getEnvVar :: String -> Env String
getEnvVar var = do
    env <- ask
    pure . fromJust . lookup var $ env
```

And the whole working example looks like this:

main :: IO ()
main = do
    env <- parseEnv <$> readFile ".env"
    runReader sendReminderToPayInvoice env

parseEnv :: String -> Map String String
parseEnv str = fromList $ map ((\(k, v) -> (k, drop 1 v)) . break (== '=')) (lines str)

sendReminderToPayInvoice :: Env (IO ())
sendReminderToPayInvoice = do
    debtors <- getDebtorsInfo
    pure $ do
        print "We sent emails to:"
        debtors >>= print

getDebtorsInfo :: Env (IO [User])
getDebtorsInfo = do
    users <- getUsersInfo
    pure $ filter (\u -> isPaid u && hasDebt u) <$> users

getUsersInfo :: Env (IO [User])
getUsersInfo = do
    let table = "public.users"
    callDatabase table

callDatabase :: String -> Env (IO [User])
callDatabase table = do
    dbHost <- getEnvVar "DB_HOST"
    dbPort <- getEnvVar "DB_PORT"
    dbPass <- getEnvVar "DB_PASSWORD"
    pure $ do
        -- Fake call to the database using dbHost, dbPort, and dbPass
        putStrLn $ "Connecting to database at " ++ dbHost ++ ":" ++ dbPort
        putStrLn $ "Using password: " ++ dbPass
        putStrLn $ "Fetching data from table: " ++ table
        pure
            [ MkUser "alice" "alice@example.com" True False
            , MkUser "bob" "bob@example.com" False False
            , MkUser "charlie" "charlie@example.com" True True
            , MkUser "dave" "dave@example.com" False False
            ]

getEnvVar :: String -> Env String
getEnvVar var = do
    env <- ask
    pure . fromJust . lookup var $ env

Since the environment dependency is implicit, unless you use it like in the `getEnvVar` function, you wouldn't know about it if it wasn't for the type signature. We completely hid that dependency and cleaned it up from the code that doesn't use it.

Now, before we move on, there is one more behavior like the `ask` function that is available for reader monads, and that's the `local` function:

local :: (r -> r) -> Reader r a -> Reader r a
local f (Reader g) = Reader (g . f)

The `local` function modifies the environment before running the action. So, if I run this:

getEnvVarLocal :: String -> Env String
getEnvVarLocal var = local (insert var "0000") $ getEnvVar var


runReader (getEnvVar      "DB_PORT") fromList [("DB_PORT", "5432")]
runReader (getEnvVarLocal "DB_PORT") fromList [("DB_PORT", "5432")]

You'll see that, no matter the value that is actually on the environment, the `getEnvVarLocal` function will always return the same value because the `local` function replaces the environment.

This could be useful in certain situations.

Finally, let's talk about the `State` monad.

# The `State` of things

The final monand we'll explore this lesson is the `State` monad. This monad is sort of the spiritual evolution of the `Reader` mondad, since we want an effect that is able not only able to read but also write to a state.

The example we'll use for this is a vending machine. If you've ever used a traditional vending machine in the past, you'll know how they work. They take coins, and at any point in time you can ask for the item. If the value of the coins is enough, you'll get the item and the change. Else, you won't but you can ask for the change back.

Ok, so, let's model this business logic. Starting with modelling our domain:

In [ ]:
data VendingState = MkVendingState { items :: Int, credit :: Int } deriving Show

We have a pretty simple domain here. We have the `VendingState` type that  represents the state of the vending machine. Inside, it has the amount of `items` inside the machine as an `Int`, and the `credit` or "value" added by the user that wants an item also as an `Int`. So, at any point in time, we have to know how many items are still inside the vending machine and how much value did the user put inside via coins.

Now, we can present the three ways of changing the state of a vending machine. The first change of state is when a user inserts a coin:

In [ ]:
insertCoin :: Int -> VendingState -> ((), VendingState)
insertCoin amount vmState = ((), vmState { credit = credit vmState + amount })

The `insertCoin` function takes the value of the coin and the current state of the vending machine as parameters, and returns a tuple containing the result of the operation, which in this case is `()`, and the new state of the vending machine. If this was the only function, we could've drop the tuple and just return the vending machine state. However, this is part of several functions that need to work together in any order, and the other functions will return results besides the new state. So, we have to adapt our return type to accomodate that.

Regarding the actual implementation, the only thing we do is to add the coin amount to whaterver amount was in the previous state and assign that as the credit of the new state. Since we're not touching the amount of items, that remains the same as the initial state.

Now, let's create the function that performs the vendinding action:

In [ ]:
-- Attempts to vend an item. Price is, say, 25 units.
vend :: VendingState -> (String, VendingState) -- Returns a message and new state
vend vmState
  | items vmState <= 0 = ("Out of stock", vmState)
  | credit vmState < 25 = ("Not enough credit. Price: 25", vmState)
  | otherwise = ("Item vended", vmState { items = items vmState - 1, credit = credit vmState - 25 })

The `vend` function takes the current vending state and tries to vend an item. We chose to prize all the items 25 coins. In a real scenario, we could read the prize from a database, but we went the simple way. So, ther are three possible results:
- If there are no items, the result is that we're out of stock, and return the old state back.
- If there are but the credit is below the prize point, the result is that there's not enough credit, and return the old state back.
- In any other case, ther are items and we have enough credit, so the result is that we vend the item and update the stat to have one item and 25 credit less than the previous state.

Finally, we create the function to get the change back in case we put more than 25 credits worth of coins:

In [ ]:
getChange :: VendingState -> (Int, VendingState)
getChange vmState = (credit vmState, vmState { credit = 0 })

The `getChnange` function takes the current vending machine's state, returns as a result all the credit inside, and updates the state to have no credits. Again, in this case the items remain untouched.

Now that we have the basic building blocks let's simulate a bunch of actions:

In [ ]:
manualVendingSequence :: VendingState -> (String, VendingState)
manualVendingSequence initialVmState =
    let ((), vmState1) = insertCoin 10 initialVmState
        ((), vmState2) = insertCoin 20 vmState1
        (message, vmState3) = vend vmState2
        (change, vmState4) = getChange vmState3
    in (message <> " | releasing change: " <> show change, vmState4)


manualVendingSequence $ MkVendingState 3 0 -- Initial state with 3 items and zero credits

As you can see, the `manualVendingSequence` has a single parameter: the initial state of the vending machine. Then, it performs several actions while threading the updated state from the previous step to the next every time. And, at the end, we return a tuple with the rsult and the final vending machine's state.

We start by inserting a coin worth 10 credits and get back the new `vmState1`. Since it's not enough to buy an item, we provide a second coin worth 20 credits. Make sure to provide `vmState1` instead of `initialVmState` or we'd have lost the first 10 credits!

Now we provide `vmsState2` which now has 30 credits to the `vend` function and sucessfully vend the item and get charged 25 credits for it. 

Finally, we still have 5 credits, so we provide `vmState3` as the current state to the `getChange` function to get our change back, and return a message combining the result of the third and fourth actions.

Notice how, on every step, we provide the previous state as input of the next action. This way of threading state is bad for several reasons:
- There's nothing stopping us to mistakingly passing any other state insted of the one that corresponds! 
- It's cumbersome to do.
- All the code to thread the state makes it difficult to understand the business logic that we're trying to convey!

So, now that we identified a repeating pattern that we want to get rid of, let's abstract it away! Let's take the essence a step in the sequence first:

```
manualVendingSequence initialVmState =
    let ((), vmState1) = insertCoin 10 initialVmState
    in (message <> " | releasing change: " <> show change, vmState4)
```

Here we have the first and last lines of the `manualVendingSequence`. The other three lines just repeat the first one, so, if we abstract this away, we'll be done.

Now, let's generalize a bit the names:

```
bindState state =
    let (result, newState) = action state
    in ...
```

I renamed the function to `bindState`, because we'll be binding the state throught every step. Not surprising, I know. 

Then, we generalize the action of the first line to `action`, and we provide the current `state` as parameter. We don't know what the action will do. But we know that it returns a tuple of a result and the new state, so we just name them like that: `result` and `newState`.

Now, to generalize the next line, since we have no idea how this result and new state is going ot be used, let's just name this expression `continuation`. However, since we know that this function will use both the result and the state, we'll provide both as parameters like this:

```
bindState state =
    let (result, newState) = action state
    in continuation result newState
```

Finally, since we don't have the `action` and `continuation` values inside the body of the funciton, we have to take them from somewhere, so we provide them as parameters:

In [ ]:
bindState action continuation state =
    let (result, newState) = action state
    in continuation result newState

:t bindState

And that's our final expression, which we can replace in the original function to see how it would look like if we would use it to hide the state:

In [ ]:
bindVendingSequence :: VendingState -> (String, VendingState)
bindVendingSequence =
    insertCoin 10 `bindState` \_ ->
    insertCoin 20 `bindState` \_ ->
    vend          `bindState` \message ->
    getChange     `bindState` \change ->
    returnState (message <> " | releasing change: " <> show (change :: Int))


bindVendingSequence $ MkVendingState 3 0 -- Initial state with 3 items and zero credits

Now, the cool part is that, we already know that this looks suspiciously close to a monad behavior. And, if you take a look at the type of `bindState`:

```
bindState :: (VendingState -> (a, VendingState)) -> (a -> (VendingState -> (b, VendingState))) -> (VendingState -> (b, VendingState))
(>>=)     ::               m a                   ->                (a -> m b)                  ->                m b
```

You'll notice that it's the same type signature that monad's bind operator if the type was a function that takes a state and returns a tuple of a result of any type with the state.

Cool! We just discovered the `State` monad! Now that we know how it looks like, let's implement it! As always, we start with the type, but that's the easy part, we just take whatever the `m` represents in the particular case of `bindState`, which is:

In [ ]:
newtype State s a = mkState { runState :: s -> (a, s) }

There you go! it's very close to the definition of the `Reader` monad, with the caveat that we get the input as part of the result.

Now that we have the type and we know the effect that it supposed to represent, we can start implementing the instances. The `Functor` instance looks a lot like the `Functor` instance for `Reader`:

In [ ]:
instance Functor (State s) where
--  fmap :: (a -> b) -> State s a -> State s b
    fmap f (MkState g) = MkState $ \s ->
        undefined

-- Remember:
-- f :: a -> b
-- g :: s -> (a, s)
-- result needed: MkState wrapping a function of type s -> (b, s)

Same as with `Reader`, we need to return a `State` wrapping a function. So, we can start by wrapping a funciton that takes the state, and now we have to figure out how to return the tuple of `(b, s)` with the tools we have `f`, `g`, and `s`.

This is also not very hard to come up with, we need to apply `f` to a value of type `a` to get a `b`, and the only way to get a value of type `a` is by applying `g` to the state `s`, so we do just that:

In [ ]:
instance Functor (State s) where
--  fmap :: (a -> b) -> State s a -> State s b
    fmap f (MkState g) = MkState $ \s ->
        let (a, s') = g s
        in (f a, s')

-- Remember:
-- f :: a -> b
-- g :: s -> (a, s)
-- result needed: MkState wrapping a function of type s -> (b, s)

We apply `g` to `s` to get the tuple of type `(a, s)`. But, since we already have the initial state `s`, we'll name the new state `s'`. And now that we have the value of type `a`, we can return the final tuple containing the result of applying the function `f` to the value of type `a` and the final state `s'` generated by applying the function `g` to the origninal type `s`.

I'll leave checking the Functor laws as homework and move ont to the Applicative instance:

In [ ]:
instance Applicative (State s) where
--  pure :: a -> State s a
    pure x = undefined

--  (<*>) :: State s (a -> b) -> State s a -> State s b
    (MkState f) <*> (MkState g) = undefined

-- Remember:
-- f :: s -> a -> (b, s)
-- g :: s -> (a, s)
-- result needed for (<*>): MkState wrapping a function of type s -> (b, s)

We already said that the effect is to use or alter the state. So, for the implementation of `pure` we just need to ignore the state. However, we're required by the type to provide the state inside the tuple that we return, so the only option for us it so take a state and pass it along without modifying it. So, the clearest implementation for this case is this one:

In [ ]:
instance Applicative (State s) where
--  pure :: a -> State s a
    pure x = MkState $ \s -> (x, s)

--  (<*>) :: State s (a -> b) -> State s a -> State s b
    (MkState f) <*> (MkState g) = undefined

-- Remember:
-- f :: s -> a -> (b, s)
-- g :: s -> (a, s)
-- result needed for (<*>): MkState wrapping a function of type s -> (b, s)

`pure` takes a value and lifts it to the `State` Applicative by injecting it inside the State structure but without interacting with the state whatsoever.

Now, let's see how we can tackle the `<*>` operator. This one is also pretty straight forward if you follow the types, with a few caveats:

In [ ]:
instance Applicative (State s) where
--  pure :: a -> State s a
    pure x = undefined

--  (<*>) :: State s (a -> b) -> State s a -> State s b
    (MkState f) <*> (MkState g) = MkState $ \s ->
        let (f', s')  = f s
            (a, s'') = g s'
        in  (f' a, s'')

-- Remember:
-- f :: s -> (a -> b, s)
-- g :: s -> (a, s)
-- result needed for (<*>): MkState wrapping a function of type s -> (b, s)

Since both `f` and `g` have to be applied to a value of type `s`, by only following the types, we coul have implemented this differently by applying `g` the original `s` and then `f` to the new `s'` state generated by `g`. However, we did it the other way around, why is that? Because we understand the intent behind the `<*>` opearator. When we learned about Applicative, we learned that this operator allows us to sequence effects in the order we write them. And because we know that, we know that we have to run the effect of `f`, the first function, before the effect of `g`, the second function.

Once we have the final expressions, we apply ghe funcion `f' :: a -> b` to the value of type `a` and return it inside a tuple with the final state `s''`.

Finally, we implement the Monad instance by using the same logic we defined in the `bindState` function but in a more general way:

In [ ]:
instance Monad (State s) where
--  (>>=) :: State s a -> (a -> State s b) -> State s b
    (MkState g) >>= f = undefined

-- Remember:
-- g :: s -> (a, s)
-- f :: a -> (s -> (b, s))
-- result needed for (>>=): MkState wrapping a function of type s -> (b, s)

We start by returning a function that takens the state `s` wrapped in a `MkState` constructor:

In [ ]:
instance Monad (State s) where
--  (>>=) :: State s a -> (a -> State s b) -> State s b
    (MkState g) >>= f = MkState $ \s ->
        undefined

-- Remember:
-- g :: s -> (a, s)
-- f :: a -> (s -> (b, s))
-- result needed for (>>=): MkState wrapping a function of type s -> (b, s)

Now, we have to figure out a way to return a state containing `b`:

In [ ]:
instance Monad (State s) where
--  (>>=) :: State s a -> (a -> State s b) -> State s b
    (MkState g) >>= f = MkState $ \s ->
        let (a, s') = g s
            MkState h = f a
        in h s'

-- Remember:
-- g :: s -> (a, s)
-- f :: a -> (s -> (b, s))
-- h :: s -> (b, s)
-- result needed for (>>=): MkState wrapping a function of type s -> (b, s)

 Our only choice to get a value of type `a` is to apply `g` to the initial state `s`. 

Once we do that, we can apply `f` to the resulting `a` to get a value of type `State s b`. However we are already inside a `MkState` constructor, so what we really want is the tuple of `b` with the latest state `s'`, so, what we do is pattern match to extract the function `h` and apply it to the latest `s'`. That would return a new tuple containing the value of type `b` and a final state that we can provide as the final result of the expression.

These instances might appear complicated the first time you see them. But, thanks to the guidance of GHC, you don't have to figure the hole thing in your head, you just have to solve each type error one by one moving each step closer and closer to the final expression.

Now that we have our `State` monad, let's reimplement the example using it! Starting with the `insertCoin` functoin that was previously defined like this:

In [ ]:
insertCoin :: Int -> VendingState -> ((), VendingState)
insertCoin amount vmState = ((), vmState { credit = credit vmState + amount })

Now that we can thread the state, we can replace this function. Or can we? 

It turns out that, similar to how we would provide the environment value in the `Reader` monad but we had to define the function `ask` to actually get the value, we have to add a few helper function to make our lives easier when dealing with the `State` monad. More concretely, for the `Reader` monad we only needed to `ask` the reader value, but in this case, we have to be able to `modify` the state, `put` new states, and `get` the current state.


We can modify the state like this:

In [ ]:
-- Helper function to modify the state
modify :: (s -> s) -> State s ()
modify f = MkState $ \s -> ((), f s)

This is sort of like the opposite of an `fmap`.We modify the state, the structure, instead of the value. And to do that, we take a function that modified the state as parameter and create a new state where we take the old state and apply the funciton to it to return the new state. In this case, we return `()` as the first value of the tuple because we don't care about the result, we only care about the state.

Now that we have this funciton, we can use it to create our new `insertCoin` function:

In [ ]:
insertCoin :: Int -> State VendingState ()
insertCoin amount = modify $ \state -> state { credit = credit state + amount }

As you can see, the funciton we provide to the `modify` helper function takes the old state and changes its `credit` field by adding the `amount` provided as parameter.

You can also see that the type of the returned value changed to a `State` monad that holds the `VendingState` state and that returns a `()`.

Now, the next function was `vend`:

In [ ]:
vend :: VendingState -> (String, VendingState) -- Returns a message and new state
vend state
  | items state <= 0 = ("Out of stock", state)
  | credit state < 25 = ("Not enough credit. Price: 25", state)
  | otherwise = ("Item vended", state { items = items state - 1, credit = credit state - 25 })

In this case, we will have to both access the state, to compare the values, and if everything is fine, we'll need to put a new state. So, we'll define these two helper functions:

In [ ]:
-- Helper function to get the current state
get :: State s s
get = MkState $ \s -> (s, s)

-- Helper function to put a new state
put :: s -> State s ()
put s = MkState $ \_ -> ((), s)

The `get` function takes a state `s` and provides it both as the state and as the result. That way, we can use the usual Monad mechanics to run the effect and get the state as result. Virtually the same as what we did with the `Reader` monad.

In the case of `put`, we take a new `s` state as parameter, ignore the one provided by the type and put that state as the new state. That way, we can replace the stat with a brand new one. We also return the `()` value as result in this case since we don't care about the result, only about the effect.

Now that we have these helper functions, we can modify the `vend` function to use the `State` monad like this:

In [ ]:
vend :: State VendingState String
vend = do
    state <- get
    case (items state <= 0, credit state < 25) of
        (True, _) -> return "Out of stock"
        (_, True) -> return "Not enough credit. Price: 25"
        _ -> do
            put $ state { items = items state - 1, credit = credit state - 25 }
            pure "Item vended"

Since we are in side the `State` monad, we can use `do` notation. So, we create a `do` block and get the value of the current state using the `get` helper function.

Then, we create a tuple with the two conditions we want to check for and pattern match for each `True` value to return the correct message in case something is wrong. If none of the two values is `True`, it means we can vend the item, so we modify the old state to have one item and 25 credits less than the original one, and put that new state as the actual vending machine state using the `put` helper funciton. At the end we lift a string with a successfull message.

Finally, we modify the `getChange` function to use our new `State` monad like this:

In [ ]:
getChange :: State VendingState Int
getChange = do
    state <- get
    put $ state { credit = 0 }
    pure $ credit state

We also need to `get` the curent state to know how many credits to return. And after we do that, we reset the credits by setting a new state with zero credits and same items as the previous state. The las line returns the `credits` in the previous `state` that we got before the modification and return that.

And that's it! Now that we refactored the whole example to use the `State` monad, we can enjoy the fruits of our hard work by re-implementing the same vending sequence as before. To highlight the results, this is how it was at the beginning:

manualVendingSequence :: VendingState -> (String, VendingState)
manualVendingSequence initialVmState =
    let ((), vmState1) = insertCoin 10 initialVmState
        ((), vmState2) = insertCoin 20 vmState1
        (message, vmState3) = vend vmState2
        (change, vmState4) = getChange vmState3
    in (message <> " | releasing change: " <> show change, vmState4)


manualVendingSequence $ MkVendingState 3 0 -- Initial state with 3 items and zero credits

And this is how it looks now:

In [ ]:
vendingSequence :: State VendingState String
vendingSequence = do
    insertCoin 10
    insertCoin 20
    message <- vend
    change <- getChange
    return $ message <> " | releasing change: " <> show change
    

runState vendingSequence $ MkVendingState 3 0 -- Initial state with 3 items and zero credits

Waaaaaaay better! The `vendingSequence` function:
- Clearly shows the business logic, since there's no boilerplate in sight.
- Doesn't allow for mistakes while threading the state because that is done automatically by the `State` monad.
- Is easy to read and write.

I'd cal this a rotund success!

And with this, I declare the last lecture of the course before the final project finished. As your last homework before the final project, try to prove that all these instances follow the `Functor`, `Applicative`, and `Monad` laws. Ohter than that, prepare yourself to write a lot of code with me in the next few and final lectures where we'll implement a real-world program.

As always, make sure to do your homework, and I'll see you in the next one!